In [ ]:
%matplotlib notebook
import pykep as pk
from matplotlib import pyplot as plt
import numpy as np

import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from pykep.orbit_plots import plot_planet, plot_lambert
from pykep.planet import jpl_lp
from pykep import epoch, DAY2SEC, SEC2DAY, AU, MU_SUN, lambert_problem


In [ ]:
# planets
mars = jpl_lp('mars')
earth = jpl_lp('earth')

In [ ]:
# times
t1 = epoch(0)
t2 = epoch(640)
dt = (t2.mjd2000 - t1.mjd2000) * DAY2SEC

In [ ]:
fig = plt.figure()
axis = fig.gca(projection='3d')

# plotting the planets
plot_planet(earth, t0=t1, color=(0.8, 0.8, 1), legend=True, units=AU, ax=axis)
plot_planet(mars, t0=t2, color=(0.8, 0.8, 1), legend=True, units=AU, ax=axis)
axis.set_xlim(-1.5, 1.5)
axis.set_ylim(-1.5, 1.5)
axis.set_zlim(-1.5, 1.5)

# boundary conditions
rE, vE = earth.eph(t1)
rM, vM = mars.eph(t2)

# Lambert's problem
l = lambert_problem(rE, rM, dt*2, MU_SUN, False, 2)
plot_lambert(l, sol=1, color='b', legend=True, units=AU, ax=axis)

In [ ]:
# calculating a bunch of lambert's problems with fixed start date, varying transfer times

res = 100



solutions = []
solutions2 = []
transfer_times = np.arange(dt/10, dt*4, (dt*2-dt/2)/res)
# for t in transfer_times:
#     rM, vM = mars.eph(epoch(t1.mjd2000 + t * SEC2DAY))

start_times = np.arange(0, 2000, .1)
for start_offset in start_times:
    t1_offset = epoch(t1.mjd2000 + start_offset)
    rE, vE = earth.eph(t1_offset)
    rM, vM = mars.eph(epoch(t1.mjd2000 + dt * SEC2DAY))
    vE, vM = np.array(vE), np.array(vM)

#     l = lambert_problem(rE, rM, t, MU_SUN, False, 1)
    l = lambert_problem(rE, rM, dt, MU_SUN, False, 0)
    a_am = np.array(l.get_x())**2 +1 
    delta_v = np.linalg.norm(np.array(l.get_v1())-vE, axis=1) + np.linalg.norm(np.array(l.get_v2()-vM), axis=1)
#     solutions.append([t, a_am, delta_v])
    solutions.append([t1_offset.mjd2000, a_am, delta_v])
    


In [ ]:
results2 = []

res = 150
transfer_times = np.linspace(dt/10, dt*1, res)
start_times = np.linspace(250, 750, res)
delta_vs = np.zeros([res, res])

for i, start_time in enumerate(start_times):
    t1_offset = epoch(t1.mjd2000 + start_time)
    rE, vE = earth.eph(t1_offset)
    
    for j, transfer_time in enumerate(transfer_times):
        t2 = epoch(t1_offset.mjd2000 + transfer_time * SEC2DAY)
        rM, vM = mars.eph(t2)
        
        l = lambert_problem(rE, rM, transfer_time, MU_SUN, False, 0)
        delta_v = np.linalg.norm(np.array(l.get_v1())-vE, axis=1) + np.linalg.norm(np.array(l.get_v2())-vM, axis=1)

        results2.append([t1_offset.mjd2000, t2.mjd2000, delta_v[0]])
        delta_vs[j, i] = delta_v

In [ ]:
# reorganizing transfer_time - dv pairs for different number of revolutions
dv = []
for solution in solutions:
    if len(dv) < len(solution[1]):
        for i in range(len(solution[1]) - len(dv)):
            dv.append([[],[]])
    t = solution[0]
    delta_vs = solution[2]
    for i, delta_v in enumerate(delta_vs):
        dv[i][1].append(delta_v)
        dv[i][0].append(t)


In [ ]:
fig = plt.figure(2)
ax = fig.add_subplot(1, 1, 1)
# for solution in solutions:
#     t = solution[0]
#     delta_vs = solution[1]
#     for delta_v in delta_vs:
#         ax.plot(t, delta_v, 'k.')
for x in dv:
    ax.plot(x[0], x[1], '.')

In [ ]:
fig = plt.figure(3)
ax = fig.add_subplot(1, 1, 1, projection='3d')
# for result in results2:
    
#     ax.scatter(result[0], result[1], result[2], '*', color='k')

xx, yy = np.meshgrid(range(res), range(res))
ax.plot_surface(xx, yy, delta_vs)

# results2 = np.array(results2)
# ax.contourf(start_times, transfer_times, delta_vs, np.min(results2[:, 2]) * np.array([ 1.01, 1.011,1.05, 1.1, 2., 3., 4., 5.,]) )
# plt.xlabel('x')
# plt.ylabel('y')


In [ ]:
len(results2)

In [ ]:
np.arange(res)

In [ ]:
delta_vs.shape